In [ ]:
import logging
import sys

fx_spot_tickers = [
    "EURUSD=X",
    "GBPUSD=X",
    "JPYUSD=X",
    "AUDUSD=X",
    "NZDUSD=X",
    "USDCHF=X",
    "USDCAD=X",
    "USDSEK=X",
    "USDNOK=X",
]
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

In [ ]:
import pickle as pkl
from importlib.resources import files

import yfinance as yf

import backtest_lib.examples

# data = yf.download(
#     tickers=fx_spot_tickers,
#     start="2020-01-01",
#     end="2025-01-01",
#     interval="1d",
#     auto_adjust=True,
# )

data = pkl.load(files(backtest_lib.examples).joinpath("fx_spot.pkl").open("rb"))

pkl.dump(data, files(backtest_lib.examples).joinpath("fx_spot.pkl").open("wb"))

In [ ]:
import backtest_lib as btl

px_flat = data["Close"].rename_axis(index="date", columns=None).ffill().reset_index()

market_view = btl.MarketView(prices=px_flat)

In [ ]:
universe = tuple(fx_spot_tickers)


def index_strategy(universe, current_portfolio, market, ctx):
    return btl.Decision(current_portfolio)

In [ ]:
from backtest_lib.backtest import Backtest
from backtest_lib.portfolio import uniform_portfolio

backtest = Backtest(
    market_view=market_view,
    universe=universe,
    strategy=index_strategy,
    initial_portfolio=uniform_portfolio(universe),
)

In [ ]:
results = backtest.run()

In [ ]:
results.asset_returns.by_period

In [ ]:
results.asset_returns.by_period.as_df(show_securities=True)

In [ ]:
results.asset_returns.by_security[["AUDUSD=X", "EURUSD=X"]].after(
    "2023-01-01"
).by_period[:5].by_security.to_dataframe()

In [ ]:
import altair as alt

alt.theme.enable("dark")

In [ ]:
results.asset_returns.by_security["EURUSD=X"].plot.line()

In [ ]:
import altair as alt

results.holdings.between("2020-01-01", "2026-01-01").by_security["EURUSD=X"].plot.line(
    y_padding=0.01, color="red", smoothing=20
) + results.holdings.between("2020-01-01", "2026-01-01").by_security[
    "GBPUSD=X"
].plot.line(y_padding=0.01, color="steelblue", smoothing=20)

In [ ]:
(
    results.asset_returns.by_security["GBPUSD=X"].plot.hist(bins=20)
    + results.asset_returns.by_security["EURUSD=X"].plot.hist()
)

In [ ]:
(
    results.asset_returns.by_security["GBPUSD=X"].plot.kde(color="red")
    + results.asset_returns.by_security["EURUSD=X"].plot.kde()
).properties(width=1000)

In [ ]:
results.asset_returns.between("2020-01-01", "2020-02-01").by_security[
    "GBPUSD=X"
].plot.bar()